Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
pickle_file = '../lesson09/notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save  # hint to help gc free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
    dataset = dataset.reshape((-1, image_size, image_size, num_channels)).astype(np.float32)
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [4]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
        / predictions.shape[0])

**Calculating Output Size**

$OutSize=\frac{(InSize−FilterSize+2PaddingSize)}{Stride}+1$

+ PaddingSize
  - 'SAME' = -1
  - 'VALID' = 0

In [5]:
# Create image size function based on input, filter size, padding and stride
# 2 convolutions only
def calc_output_size(input_size, filter_size, stride, padding_type):
    if padding_type == 'SAME':
        padding_size = -1.00
    elif padding_type == 'VALID':
        padding_size = 0.00
    else:
        padding_size = 0.00
    # Conv
    output = float(((input_size - filter_size - 2*padding_size) / stride) + 1.00)
    return int(np.ceil(output))

def calc_output_size_loop(loops, input_size, filter_size, stride, padding):
    output_size = input_size
    for i in range(loops):
        output_size = calc_output_size(output_size, filter_size, stride, padding)
    return output_size

def calc_output_size_with_pool_loop(loops, input_size, conv_filter_size, conv_stride, pool_filter_size, pool_stride, padding):
    output_size = input_size
    for i in range(loops):
        output_size = calc_output_size(output_size, conv_filter_size, conv_stride, padding)
        output_size = calc_output_size(output_size, pool_filter_size, pool_stride, padding)
    return output_size

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [6]:
batch_size = 16
patch_size = 5    # filter size
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

    '''Input data.'''
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    '''Variables.'''
    # Convolution 1 Layer
    # [filter_height, filter_width, in_channels, out_channels] = [5,5,1,16]
    layer1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))

    # Convolution 2 Layer
    # [filter_height, filter_width, in_channels, out_channels] = [5,5,16,16]
    layer2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    
    # Fully Connected Layer (Densely Connected layout)
    # Use neurons to allow processing of entire image
    final_image_size = calc_output_size_loop(2, image_size, patch_size, 2, padding='SAME')
    layer3_weights = tf.Variable(tf.truncated_normal([final_image_size * final_image_size * depth, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    
    # Readout Layer: Softmax Layer
    layer4_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
    '''Model.'''
    def model(data):
        # First Convolutional Layer
        conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer1_biases)
        # Second Convolutional Layer
        conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer2_biases)
        # Full Connected Layer
        shape = hidden.get_shape().as_list()
        reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        # Readout layer: Softmax Layer
        return tf.matmul(hidden, layer4_weights) + layer4_biases
  
    '''Training computation.'''
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
    '''Optimizer.'''
    # Learning rate of 0.05
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
    '''Predictions for the training, validation, and test data.'''
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [7]:
num_steps = 1001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 100 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.709522
Minibatch accuracy: 6.2%
Validation accuracy: 10.0%
Minibatch loss at step 100: 0.907778
Minibatch accuracy: 68.8%
Validation accuracy: 68.7%
Minibatch loss at step 200: 0.767462
Minibatch accuracy: 75.0%
Validation accuracy: 77.6%
Minibatch loss at step 300: 0.383572
Minibatch accuracy: 87.5%
Validation accuracy: 79.5%
Minibatch loss at step 400: 0.237802
Minibatch accuracy: 93.8%
Validation accuracy: 80.3%
Minibatch loss at step 500: 0.651773
Minibatch accuracy: 87.5%
Validation accuracy: 81.0%
Minibatch loss at step 600: 0.303047
Minibatch accuracy: 93.8%
Validation accuracy: 81.3%
Minibatch loss at step 700: 0.921499
Minibatch accuracy: 68.8%
Validation accuracy: 82.3%
Minibatch loss at step 800: 0.637393
Minibatch accuracy: 81.2%
Validation accuracy: 82.7%
Minibatch loss at step 900: 0.584981
Minibatch accuracy: 81.2%
Validation accuracy: 83.0%
Minibatch loss at step 1000: 0.447596
Minibatch accuracy: 87.5%
Validation accuracy: 82.6%


---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

**Details**
+ image_size = 28
+ Convolutions
  - conv_filter_size = 5
  - conv_stride = 1
+ Max Pooling
  - pool_filter_size = 2
  - pool_stride = 2
+ padding='SAME'
+ Layers
  - Convolution
  - Convolution
  - Fully-connected
  - Readout

In [8]:
batch_size = 16
patch_size = 5    # filter size
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

    '''Input data.'''
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    '''Variables.'''
    # Convolution 1 Layer
    # [filter_height, filter_width, in_channels, out_channels] = [5,5,1,16]
    layer1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))

    # Convolution 2 Layer
    # [filter_height, filter_width, in_channels, out_channels] = [5,5,16,16]
    layer2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    
    # Fully Connected Layer (Densely Connected layout)
    # Use neurons to allow processing of entire image
    final_image_size = calc_output_size_with_pool_loop(loops=2, input_size=image_size, conv_filter_size=patch_size,
                                                       conv_stride=1, pool_filter_size=2, pool_stride=2, padding='SAME')
    layer3_weights = tf.Variable(tf.truncated_normal([final_image_size * final_image_size * depth, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    
    # Readout Layer: Softmax Layer
    layer4_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
    '''Model.'''
    def model(data):
        # First Convolutional Layer
        conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer1_biases)
        pool = tf.nn.max_pool(hidden, [1,2,2,1], [1,2,2,1], padding='SAME')
        # Second Convolutional Layer
        conv = tf.nn.conv2d(pool, layer2_weights, [1, 1, 1, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer2_biases)
        pool = tf.nn.max_pool(hidden, [1,2,2,1], [1,2,2,1], padding='SAME')
        # Full Connected Layer
        shape = pool.get_shape().as_list()
        reshape = tf.reshape(pool, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        # Readout layer: Softmax Layer
        return tf.matmul(hidden, layer4_weights) + layer4_biases
  
    '''Training computation.'''
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels,logits=logits))
    
    '''Optimizer.'''
    # Learning rate of 0.05
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
    '''Predictions for the training, validation, and test data.'''
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [9]:
num_steps = 1001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 100 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.681913
Minibatch accuracy: 18.8%
Validation accuracy: 10.0%
Minibatch loss at step 100: 1.323123
Minibatch accuracy: 62.5%
Validation accuracy: 50.7%
Minibatch loss at step 200: 0.950998
Minibatch accuracy: 75.0%
Validation accuracy: 75.8%
Minibatch loss at step 300: 0.503479
Minibatch accuracy: 81.2%
Validation accuracy: 79.3%
Minibatch loss at step 400: 0.326552
Minibatch accuracy: 93.8%
Validation accuracy: 80.1%
Minibatch loss at step 500: 0.636770
Minibatch accuracy: 87.5%
Validation accuracy: 80.9%
Minibatch loss at step 600: 0.312628
Minibatch accuracy: 87.5%
Validation accuracy: 82.1%
Minibatch loss at step 700: 0.711972
Minibatch accuracy: 68.8%
Validation accuracy: 81.9%
Minibatch loss at step 800: 0.740429
Minibatch accuracy: 75.0%
Validation accuracy: 83.2%
Minibatch loss at step 900: 0.595264
Minibatch accuracy: 87.5%
Validation accuracy: 83.7%
Minibatch loss at step 1000: 0.460425
Minibatch accuracy: 87.5%
Validation accuracy: 82.6%

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

**Details**
+ image_size = 28
+ Convolutions
  - conv_filter_size = 5
  - conv_stride = 1
+ Max Pooling
  - pool_filter_size = 2
  - pool_stride = 2
+ padding='VALID'
+ Prevent overfitting
  - <font color=red>Learning rate decay</font>
  - Regularization
  - <font color=red>Dropout</font>
+ Layers
  - Convolution
  - Pooling
  - Convolution
  - Pooling
  - Fully-connected
  - Readout

In [12]:
batch_size = 16
patch_size = 5    # filter size
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

    '''Input data.'''
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    '''Variables.'''
    # Convolution 1 Layer
    # [filter_height, filter_width, in_channels, out_channels] = [5,5,1,16]
    layer1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))

    # Convolution 2 Layer
    # [filter_height, filter_width, in_channels, out_channels] = [5,5,16,16]
    layer2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    
    # Fully Connected Layer (Densely Connected layout)
    # Use neurons to allow processing of entire image
    final_image_size = calc_output_size_with_pool_loop(loops=2, input_size=image_size, conv_filter_size=patch_size,
                                                       conv_stride=1, pool_filter_size=2, pool_stride=2, padding='SAME')
    layer3_weights = tf.Variable(tf.truncated_normal([final_image_size * final_image_size * depth, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    
    # Readout Layer: Softmax Layer
    layer4_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
    '''Model.'''
    def model(data):
        # First Convolutional Layer
        conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer1_biases)
        # Dropout
        hidden = tf.nn.dropout(hidden, 0.5)
        pool = tf.nn.max_pool(hidden, [1,2,2,1], [1,2,2,1], padding='SAME')
        # Second Convolutional Layer
        conv = tf.nn.conv2d(pool, layer2_weights, [1, 1, 1, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer2_biases)
        # Dropout
        hidden = tf.nn.dropout(hidden, 0.5)
        pool = tf.nn.max_pool(hidden, [1,2,2,1], [1,2,2,1], padding='SAME')
        # Full Connected Layer
        shape = pool.get_shape().as_list()
        reshape = tf.reshape(pool, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)

        # Readout layer: Softmax Layer
        return tf.matmul(hidden, layer4_weights) + layer4_biases
  
    '''Training computation.'''
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels,logits=logits))
    
    '''Optimizer.'''
    # Decaying learning rate
    global_step = tf.Variable(0, trainable=False)
    learning_rate = tf.train.exponential_decay(0.1, global_step, 100000, 0.96, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
    '''Predictions for the training, validation, and test data.'''
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [13]:
num_steps = 1001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 100 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 4.163013
Minibatch accuracy: 18.8%
Validation accuracy: 8.3%
Minibatch loss at step 100: 1.542737
Minibatch accuracy: 37.5%
Validation accuracy: 51.2%
Minibatch loss at step 200: 1.407900
Minibatch accuracy: 56.2%
Validation accuracy: 73.2%
Minibatch loss at step 300: 0.344960
Minibatch accuracy: 87.5%
Validation accuracy: 78.9%
Minibatch loss at step 400: 0.344542
Minibatch accuracy: 93.8%
Validation accuracy: 78.1%
Minibatch loss at step 500: 0.519524
Minibatch accuracy: 87.5%
Validation accuracy: 80.0%
Minibatch loss at step 600: 0.503457
Minibatch accuracy: 87.5%
Validation accuracy: 80.3%
Minibatch loss at step 700: 0.767588
Minibatch accuracy: 68.8%
Validation accuracy: 82.1%
Minibatch loss at step 800: 0.583797
Minibatch accuracy: 75.0%
Validation accuracy: 81.7%
Minibatch loss at step 900: 0.719865
Minibatch accuracy: 87.5%
Validation accuracy: 81.6%
Minibatch loss at step 1000: 0.215697
Minibatch accuracy: 93.8%
Validation accuracy: 82.5%
